### **Optimization using Pyomo - LP and NLP**

Let's start with a purely mathematical example (Non-linear Program):

$$\begin{align*} \min_{x} \quad & x_1^2 + 2 x_2^2 - x_3 \\
\mathrm{s.t.} \quad & x_1 + x_2 = 1 \\
& x_1 + 2 x_2 - x_3 = 5 \\
& -10 \leq x_1, x_2, x_3 \leq 10 
\end{align*} $$

We want to solve the constrained optimization problem numerically.

In [1]:
import pyomo.environ as pe
import pyomo.opt as po
import pandas as pd

In [2]:
# Create instance of concrete pemo model.
# concrete means all of the sets and model data are specified at the time of model construction.
# In this class, you'll use a concrete model.
m = pe.ConcreteModel()

## Declare variables with initial values with bounds
bnds = (-10, 10)
m.x1 = pe.Var(bounds=bnds)
m.x2 = pe.Var(bounds=bnds)
m.x3 = pe.Var(bounds=bnds)

## Declare objective
m.OBJ = pe.Objective(expr=m.x1**2 + 2*m.x2**2 - m.x3, sense = pe.minimize)

## Declare equality constraints
m.con1 = pe.Constraint(expr= m.x1 + m.x2 == 1)
m.con2 = pe.Constraint(expr= m.x1 + 2*m.x2 - m.x3 == 5)


## Display model
m.pprint()

3 Var Declarations
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :   -10 :  None :    10 : False :  True :  Reals
    x2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :   -10 :  None :    10 : False :  True :  Reals
    x3 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :   -10 :  None :    10 : False :  True :  Reals

1 Objective Declarations
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x1**2 + 2*x2**2 - x3

2 Constraint Declarations
    con1 : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :   1.0 : x1 + x2 :   1.0 :   True
    con2 : Size=1, Index=None, Active=True
        Key  : Lower : Body           : Upper : Active
        None :   5.0 : x1 + 2*x2 - x3 :   5.0 :   True

6 Declarations: x1 x2 x3 OBJ con1 c

In [3]:
solver = po.SolverFactory('IPOPT')
results = solver.solve(m)

In [4]:
## Return the solution
print("x1 = ",pe.value(m.x1))
print("x2 = ",pe.value(m.x2))
print("x3 = ",pe.value(m.x3))
print("\n")

x1 =  0.4999999999666826
x2 =  0.5000000000333173
x3 =  -3.499999999966682




#### *Linear Programs*: Student Diet Example

You want to save money eating while remaining healthy. A healthy diet requires at least P=6 units of protein, C=15 units of carbohydrates, F=5 units of fats and V=7 units of vitamins. Due to compounding factors (blizzard during Lent), our campus only has these options:

In [5]:
df = pd.read_csv("https://ndcbe.github.io/CBE60499/data/student_diet.csv")
df.head()

,Unnamed: 0,P,C,F,V,price
0,takeaway,3.0,3,2,1,5
1,vegtables,1.0,2,0,4,1
2,bread,0.5,4,1,0,2


##### Propose an Optimization Model

In [6]:
m = pe.ConcreteModel()

**Sets**

In [7]:
nutrients = {'P', 'C', "F", "V"}
modes = {"takeaway", "vegtables", "bread"}

m.N = pe.Set(initialize=nutrients)
m.M = pe.Set(initialize=modes)

    (type: set).  This WILL potentially lead to nondeterministic behavior in
    Pyomo
    (type: set).  This WILL potentially lead to nondeterministic behavior in
    Pyomo


**Parameters**

In [8]:
price = {
    "takeaway": 5,
    "vegtables": 1,
    "bread": 2
}

m.price = pe.Param(m.M, initialize=price)

vals = {
    ('takeaway', 'P'): 3.0,
    ('takeaway', 'C'): 3.0,
    ('takeaway', 'F'): 2.0,
    ('takeaway', 'V'): 1.0,
    ('vegtables', 'P'): 1.0,
    ('vegtables', 'C'): 2.0,
    ('vegtables', 'F'): 0.0,
    ('vegtables', 'V'): 4.0,
    ('bread', 'P'): 0.5,
    ('bread', 'C'): 4.0,
    ('bread', 'F'): 1.0,
    ('bread', 'V'): 0.0
 }

m.vals = pe.Param(m.M, m.N, initialize=vals)

nr = {
    'P':6, 
    'C':15, 
    "F": 5, 
    "V": 7
}

m.nr = pe.Param(m.N, initialize=nr)


**Variables**

In [9]:
m.x = pe.Var(m.M, domain=pe.NonNegativeReals)

**Constraints**

In [10]:
def cons1(m, j):
    return sum(m.x[i]*m.vals[i, j] for i in m.M) >= m.nr[j]
m.cons = pe.Constraint(m.N, rule=cons1)

**Objective**

In [11]:
def objective(m):
    a =  sum(m.x[i]*m.price[i] for i in m.M)
    return a

m.TotCost = pe.Objective(rule=objective, sense=pe.minimize)

**Degree of Freedom Analysis**

We will later learn more about how to factor inequality constraints into degree of freedom analysis. For now, please count the number of equality and inequality constraints separately.

In [12]:
# Specify the solver
solver = po.SolverFactory('glpk')

# Solve
results = solver.solve(m)

In [13]:
for c in m.component_data_objects(pe.Var):
    print(c.name,"\t", pe.value(c))


x[takeaway] 	 1.0
x[vegtables] 	 1.5
x[bread] 	 3.0


**Take Away Messages**

Linear programs are convex. We will learn this means all local optima are global optima.
Nonlinear programs may be nonconvex. For nonconvex problems, there often existings many local optima that are not also global optima.